In [2]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

## Question 1
- Install Pipenv
- What's the version of pipenv you installed?
- Use --version to find out

In [3]:
'2024.2.0'

'2024.2.0'

## Question 2
Use Pipenv to install Scikit-Learn version 1.5.2

What's the first hash for scikit-learn you get in Pipfile.lock?

In [4]:
"6cc63febaf327f742e8bee26264e78eeccf6ffc9c25e70f3586af9a3ec0769b9"

'6cc63febaf327f742e8bee26264e78eeccf6ffc9c25e70f3586af9a3ec0769b9'

In [5]:
%pip install wget

/home/dani/.local/share/virtualenvs/05-deployment-PYyKPZJR/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Model Download
url='https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework'
!wget $url/model1.bin
!wget $url/dv.bin

--2024-10-29 05:21:53--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework/model1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 850 [application/octet-stream]
Saving to: ‘model1.bin.1’

model1.bin.1        100%[===================>]     850  --.-KB/s    in 0s      

2024-10-29 05:21:54 (20.0 MB/s) - ‘model1.bin.1’ saved [850/850]

--2024-10-29 05:21:54--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework/dv.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443...

## Question 3

Let's use these models!

* Write a script for loading these models with pickle
* Score this client:

```json
{"job": "management", "duration": 400, "poutcome": "success"}
```

What's the probability that this client will get a subscription? 

* 0.359
* 0.559
* 0.759
* 0.959

If you're getting errors when unpickling the files, check their checksum:

```bash
$ md5sum model1.bin dv.bin
3d8bb28974e55edefa000fe38fd3ed12  model1.bin
7d37616e00aa80f2152b8b0511fc2dff  dv.bin
```

In [7]:
with open('model1.bin','rb') as model_in:
    model = pickle.load(model_in)

In [8]:
with open('dv.bin','rb') as dv:
    dv = pickle.load(dv)

In [9]:
customer = {"job": "management", "duration": 400, "poutcome": "success"}

In [14]:
X = dv.transform(customer)
prediction = model.predict_proba(X)
prediction[0,1]

np.float64(0.7590966516879658)

## Question 4

Now let's serve this model as a web service

* Install Flask and gunicorn (or waitress, if you're on Windows)
* Write Flask code for serving the model
* Now score this client using `requests`:

```python
url = "YOUR_URL"
client = {"job": "student", "duration": 280, "poutcome": "failure"}
requests.post(url, json=client).json()
```

What's the probability that this client will get a subscription?

* 0.335
* 0.535
* 0.735
* 0.935

In [20]:
import requests

In [24]:
response = requests.post(url='http://0.0.0.0:9696/predict',json={
    "job": "student", "duration": 280, "poutcome": "failure"
})

response.json()

{'churn': False, 'prediction': 0.33480703475511053}

## Docker

Install [Docker](https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/05-deployment/06-docker.md). 
We will use it for the next two questions.

For these questions, we prepared a base image: `svizor/zoomcamp-model:3.11.5-slim`. 
You'll need to use it (see Question 5 for an example).

This image is based on `python:3.11.5-slim` and has a logistic regression model 
(a different one) as well a dictionary vectorizer inside. 

This is how the Dockerfile for this image looks like:

```docker 
FROM python:3.11.5-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]
```

We already built it and then pushed it to [`svizor/zoomcamp-model:3.11.5-slim`](https://hub.docker.com/r/svizor/zoomcamp-model).

> **Note**: You don't need to build this docker image, it's just for your reference.


## Question 5

Download the base image `svizor/zoomcamp-model:3.11.5-slim`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 45 MB
* 130 MB
* 245 MB
* 330 MB

You can get this information when running `docker images` - it'll be in the "SIZE" column.


## Dockerfile

Now create your own Dockerfile based on the image we prepared.

It should start like that:

```docker
FROM svizor/zoomcamp-model:3.11.5-slim
# add your stuff here
```

Now complete it:

* Install all the dependencies form the Pipenv file
* Copy your Flask script
* Run it with Gunicorn 

After that, you can build your docker image.



In [25]:
response = requests.post(url='http://0.0.0.0:9696/predict',json={
   "job": "management", "duration": 400, "poutcome": "success"
})

response.json()

{'churn': True, 'prediction': 0.7590966516879658}